# Combined Calibration Factor and Fold-Change Inference

In [1]:
import numpy as np
import pandas as pd
import bokeh.io
import altair as alt
import mwc.bayes
import mwc.viz
import mwc.bayes
import mwc.stats
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
# Load in the data
fluct_data = pd.read_csv('../../data/compiled_fluctuations.csv')
fc_data = pd.read_csv('../../data/compiled_fold_change.csv')

# Isolate a single carbon source. 
fluct_data = fluct_data[fluct_data['carbon']=='glucose'].copy()
fc_data = fc_data[fc_data['carbon']=='glucose'].copy()

# Compute the mean autofluorescence for each channel. 
auto = fc_data[fc_data['strain']=='auto']
mean_auto_mch = np.mean(auto['mean_mCherry'] - auto['mCherry_bg_val'])
mean_auto_yfp = np.mean(auto['mean_yfp'] - auto['yfp_bg_val'])

# Perform necessary background subtraction for fluctuation measurements. 
fluct_data['I_1_sub'] = (fluct_data['I_1'] - fluct_data['bg_val'] - mean_auto_mch) * fluct_data['area_1']
fluct_data['I_2_sub'] = (fluct_data['I_2'] - fluct_data['bg_val'] - mean_auto_mch) * fluct_data['area_2']

# Ensure positivity. 
fluct_data = fluct_data[(fluct_data['I_1_sub'] >= 0) & (fluct_data['I_2_sub'] >= 0)]

# Perform background subtraction for fluorescence measurements. 
fc_data['yfp_sub'] = np.round((fc_data['mean_yfp'] - fc_data['yfp_bg_val']) * fc_data['area_pix'])
fc_data['mch_sub'] = np.round((fc_data['mean_mCherry'] - fc_data['mCherry_bg_val']) * fc_data['area_pix'])

# Add relevant identifiers to each data set. 
fluct_data['date_idx'] = fluct_data.groupby(['date']).ngroup() + 1
fluct_data['rep_idx'] = fluct_data.groupby(['date', 'run_no']).ngroup() + 1

# Compute the means for the fold-change data
mean_fc_data = fc_data.groupby(['date', 'atc_ngml', 'run_number', 'strain']).mean().reset_index()

# Enforce positivity of YFP and mCherry.
mean_fc_data = mean_fc_data[mean_fc_data['strain'] != 'auto'].copy() # Drop unnecessary autofluorescence strain
mean_fc_data = mean_fc_data[(mean_fc_data['yfp_sub'] >= 0) & (mean_fc_data['mch_sub'] >= 0)]

# Add appropriate identifiers to mean data.
mean_fc_data['conc_idx'] = mean_fc_data.groupby(['atc_ngml', 'strain']).ngroup() + 1
mean_fc_data['rep_idx'] = mean_fc_data.groupby(['date', 'atc_ngml', 'run_number', 'strain']).ngroup() + 1

In [4]:
# Assemble the data dictionary and compile the model. 
data_dict = {'J_exp': np.max(fluct_data['rep_idx']),
             'N_fluct': len(fluct_data),
             'index_1': fluct_data['rep_idx'],
             'J_conc': np.max(mean_fc_data['conc_idx']),
             'J_conc_exp': np.max(mean_fc_data['rep_idx']),
             'N_meas': len(mean_fc_data),
             'index_2': mean_fc_data['conc_idx'],
             'I_1': fluct_data['I_1_sub'],
             'I_2': fluct_data['I_2_sub'],
             'mcherry': mean_fc_data['mch_sub'],
             'yfp': mean_fc_data['yfp_sub']}
model = mwc.bayes.StanModel('../stan/hierarchical_dilution_analysis.stan', data_dict)

Found precompiled model. Loading...
finished!


In [ ]:
samples = model.sample(iter=1000, chains=4, return_df=False)

Beginning sampling...


In [22]:
model.traceplot(varnames=['avg_rep', 'fc', 'alpha_1'])

AttributeError: 'NoneType' object has no attribute 'model_pars'

In [14]:
mean_fc_data

,date,atc_ngml,run_number,strain,area_pix,mean_yfp,mean_mCherry,fold_change,temp,yfp_bg_val,mCherry_bg_val,IPTGuM,yfp_sub,mch_sub,conc_idx,rep_idx
1,20181002,0.0,1,delta,513.222951,862.730587,197.846627,1.000000,37,167.215755,195.171007,0,356801.308197,1229.416393,1,1
2,20181002,1.0,1,dilution,546.377095,302.851303,221.376257,0.193622,37,159.584180,192.540656,0,79720.868715,12127.659218,2,2
3,20181002,2.0,1,dilution,490.318501,201.327875,217.047948,0.047401,37,153.176108,187.869161,0,23932.639344,14105.955504,3,3
4,20181002,3.0,1,dilution,500.873377,184.388816,253.726777,0.023004,37,153.073071,187.473752,0,15887.857143,32526.857143,4,4
5,20181002,4.0,1,dilution,526.977273,178.979078,281.102981,0.015212,37,153.278526,188.030942,0,13542.973485,48239.458333,5,5
6,20181002,7.0,1,dilution,509.508361,176.851273,398.209790,0.012148,37,153.085936,191.137157,0,11918.202341,103164.826087,6,6
8,20181004,0.0,1,delta,587.424000,1012.404220,190.837209,1.000000,37,162.411296,186.741542,0,492623.156800,2229.219200,1,7
9,20181004,1.0,1,dilution,558.451807,202.964154,282.338247,0.045514,37,153.184011,188.026105,0,27583.954819,52119.391566,2,8
10,20181004,2.0,1,dilution,590.298932,186.726128,292.394911,0.026366,37,152.832058,186.663100,0,20276.590747,62354.754448,3,9
11,20181004,3.0,1,dilution,555.504950,176.920856,347.404682,0.014804,37,151.739540,188.150882,0,14149.423267,87093.702970,4,10


In [17]:
mean_fc_data['rep_idx']

0      1
1      2
2      3
3      4
4      5
5      6
6      7
7      8
8      9
9     10
10    11
11    12
12    13
13    14
14    15
15    16
16    17
17    18
18    19
19    20
20    21
21    22
22    23
23    24
24    25
25    26
26    27
27    28
28    29
29    30
30    31
31    32
32    33
33    34
34    35
35    36
36    37
37    38
38    39
Name: rep_idx, dtype: int64